In [1]:
!python -m pip install astroquery > /dev/null 2>&1


In [2]:
from astroquery.jplhorizons import Horizons # Access to the solar system dynamics database
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import animation, style
from datetime import datetime, timedelta
from matplotlib.animation import writers

In order for the animation to resemble as closely as possible the reference GIF provided by the professor, the planets' time steps should not be multiples of 24 hours or 60 minutes. This ensures that the timestamps shown in each frame do not always display the same hours and/or minutes. Since the minimum allowed time unit for the step is minutes, the seconds will always be the same, but the hours and minutes will vary.

Therefore, we use a step size of **32538 minutes**, which is approximately **22.59 days**.

For the initial date, we choose **February 2, 1986**, which is also the starting date of the reference GIF. The final date corresponds to the next appearance of Halley's Comet, as stated [in this source](https://en.wikipedia.org/wiki/Halley's_Comet#Apparitions), which will be **July 28, 2061**.

In [3]:
n = 9 # Number of bodies
planetas = np.array([None]*n)
tablas = np.array([None]*n)
nombre_planetas = ["Mercury Barycenter","Venus Barycenter","Earth-Moon Barycenter","Mars Barycenter",
                   "Jupiter Barycenter","Saturn Barycenter","Uranus Barycenter","Neptune Barycenter",90000033]

for i in range(0,n):
  planetas[i] = Horizons(id=nombre_planetas[i],location="@0",epochs={"start":"1986-02-02","stop":"2061-07-28","step":"32538m"})
  tablas[i] = planetas[i].vectors()

In [4]:
# We make arrangements where we will save the X and Y positions of all the planets
X,Y = np.array([None]*n),np.array([None]*n)

for i in range(len(tablas)):
  X[i] = tablas[i]["x"].to("m").value
  Y[i] = tablas[i]["y"].to("m").value

In [6]:
# We make arrangements where we will save the dates
fechas = np.array([None]*(len(X[8])))

# Date format: Year, Month, Day, Hour, Minutes, Seconds, Milliseconds
fecha_inicio = datetime(1986,2,1)

for i in range(0,len(X[8])):
  fechas[i] = fecha_inicio
  fecha_inicio = fecha_inicio +  timedelta(minutes = 32538) # timedelta = "step" of the planets

Again, in order to generate a GIF that closely resembles the reference, we convert the `datetime` array to strings so that the printed dates display the **month name** instead of its corresponding number.


In [7]:
for i in range(0,len(fechas)):
  fechas[i] = fechas[i].strftime('%Y %b %d %H:%M:%S UT') # To print the name of the month instead of the number

Now we need to rotate the plot by 90°. The coordinates of the points after rotation are:

$$
x' = x \cos\theta - y \sin\theta \\
y' = x \sin\theta + y \cos\theta
$$

Thus, our new coordinates become:

$$
x' = -y \\
y' = x
$$

Therefore, in the first coordinates we will plot the $-y$ values, and in the second coordinates we will plot the $x$ values.


In [9]:
fig0 = plt.figure(facecolor='black')
ax0 = fig0.gca()
ax0.set_aspect("equal")
fig0.patch.set_facecolor('black')
colores = ["darkgray", "lime", "royalblue", "red", "fuchsia", "gold", "mediumturquoise", "blue", "white"]

def planetas(i):
    ax0.clear()
    plt.style.use("dark_background")
    ax0.set_facecolor('black')
    for t in range(0, n - 1):
        ax0.plot(-Y[t][i], X[t][i], "o", color=colores[t])
        ax0.plot(-Y[t][:i], X[t][:i], "-", color=colores[t])
    ax0.plot(-Y[8][i], X[8][i], "o", color=colores[8])
    ax0.plot(-Y[8], X[8], "-", color=colores[8])
    ax0.axis("off")
    ax0.set_title(fechas[i], color='white')
    ax0.set_xlim(min(-Y[8]) - 0.25e12, max(-Y[7]))
    ax0.set_ylim(min(Y[7] - 0.25e12), max(Y[7] + 0.25e12))

plt.close()
ani0 = animation.FuncAnimation(fig0, planetas, frames=len(X[8]), interval=10)
from IPython.display import HTML
ani0.save('anim.gif', writer="pillow")
HTML(ani0.to_html5_video())